In [ ]:
import csv
import math
import random
import time
from typing import List
# from matplotlib import pyplot

z_gap = 150
dist_range = [200000, 300000] # mm
line_ratio = 0.35
MAX_CONN = 18
RETRY = 10
visited = set()
start_time = int(time.time())
outliers = [
    # format:
    # ([xa, ya], [xb, yb]) meaning points a and b ain't connected
    ([467596.935572924, 178436.398350352], [468146.935572924, 177483.770406189]),
    ([458246.935572924, 175578.514517863], [458796.935572924, 174625.886573701]),
]

In [ ]:
def eq(a: float, b: float):
    return abs(a-b) < 0.001


class Point:
    def __init__(self, x: float, y: float, z: float):
        self.x = x
        self.y = y
        self.z = z
        self.ns = set()
        
    def nexts(self, prev, goal, should_line: bool):
        def h(a, b, c) -> float:
            return b.dist(a) + c.dist(b)
        res = []
        for n in self.ns:
            if n in visited:
                continue
            if prev is None or (self - prev) * (n - self) > 0: # direction constraint
                if (Point(n.x, n.y, n.z + z_gap) in visited) or (Point(n.x, n.y, n.z - z_gap) in visited):
                # check safe distance on Z axis
                    continue
                res.append(n)
        res = sorted(res, key=lambda n: h(self, n, goal))
        if not should_line:
            return res
        line_res = [n for n in res if is_line([prev, self, n])]
        return line_res + [n for n in res if n not in line_res]
        
    def is_zero(self) -> bool:
        return eq(self.x, 0) and eq(self.y, 0) and eq(self.z, 0)

    def unvisited_n(self) -> int:
        return len([n for n in self.ns if n not in visited])
    
    def is_adjacent(self, other) -> bool:
        if abs(self.z - other.z) > z_gap + 1:
            return False
        vec = self - other
        if vec.is_zero():
            return False
        if eq(self.z, other.z):
            return (vec * vec) <= (1100+1)*(1100+1)
        else:
            if eq(self.x, other.x) and eq(self.y, other.y):
                return False
            return (vec * vec) <= (1100+1)*(1100+1) + (150+1)*(150+1)

    def xy_eq(self, other) -> bool:
        return eq(self.x, other[0]) and eq(self.y, other[1])
    
    def __str__(self):
        return f"({self.x:.9f},{self.y:.9f},{self.z:.9f})"
    
    def __hash__(self) -> int:
        xi, yi, zi = int(self.x) * 10, int(self.y) * 10, int(self.z) * 10
        def cantor(x, y) -> int:     
            return (x + y) * (x + y + 1) + y * 2
        return cantor(cantor(xi, yi), zi)

    def __eq__(self, other) -> bool:
        return (self - other).is_zero()
    
    def __sub__(self, other):
        return Point(self.x - other.x, self.y - other.y, self.z - other.z)
    
    def __mul__(self, other) -> float:
        return self.x * other.x + self.y * other.y + self.z * other.z
    
    def __mod__(self, other) -> bool: # if pararrel as vectors
        return (not self.is_zero()) and (not other.is_zero()) and eq(self.x * other.y, self.y * other.x) and eq(self.x * other.z, self.z * other.x)
    
    def dist2(self, other) -> float:
        return (self - other) * (self - other)
    
    def dist(self, other) -> float:
        return math.sqrt((self - other) * (self - other))


def is_line(ps: List[Point]):
    if len(ps) <= 2:
        return True
    if ps[0] is None:
        return is_line(ps[1:])
    for i in range(0, len(ps)-2):
        if not ((ps[i+1] - ps[i]) % (ps[i+2] - ps[i+1])):
            return False
    return True
        

In [ ]:
ps = []
with open('cloud4.csv') as f:
    reader = csv.reader(f, delimiter=',')
    for row in reader:
        p = Point(*[float(x) for x in row[0:3]])
        ps.append(p)
print(len(ps))
outlier_edge = 0
for i in range(0, len(ps)):
    for j in range(i+1, len(ps)):
        if ps[i].is_adjacent(ps[j]):
            is_outlier = False
            for outlier in outliers:
                if (ps[i].xy_eq(outlier[0]) and ps[j].xy_eq(outlier[1])) or (ps[i].xy_eq(outlier[1]) and ps[j].xy_eq(outlier[0])):
                    outlier_edge += 1
                    is_outlier = True
                    break
            if not is_outlier:
                ps[i].ns.add(ps[j])
                ps[j].ns.add(ps[i])
print(f"removed {outlier_edge} outlier edges")
# pyplot.hist([len(p.ns) for p in ps], 100)
# pyplot.show()

In [ ]:
def a_star(prev: Point, cur: Point, goal: Point, path: List[Point], line_cnt: int) -> bool: # path = [start, end)
    global total
    # print(f"cur:{cur}, path length={len(path)} dist={cur.dist(goal):.1f} total={total:.1f}")
    if cur == goal:
        return True
    if int(time.time()) - start_time > 5:
        raise RuntimeError("time to bail")
    ns = cur.nexts(prev, goal, line_cnt / float(len(path)) < line_ratio - 0.2)
    if not(ns):
        return False
    for n in ns:
        visited.add(n)
        path.append(n)
        total += cur.dist(n)
        if a_star(cur, n, goal, path, line_cnt + (1 if is_line([prev, cur, n]) else 0)):
            return True
        visited.remove(n)
        path.pop()
        total -= cur.dist(n)

In [ ]:
def paths(start: Point) -> List[Point]:
    dist_th = dist_range[0] / 1.2
    very_start = start
    global visited, start_time, total
    cycle = []
    end, prev = None, None
    failed_cnt = 0
    max_conn = MAX_CONN
    while True:
        if not end:
            candidates = []
            for p in ps:
                if p in visited or p == start or p.unvisited_n() < max_conn:
                    continue
                if total > dist_th and (p - start) * (p - very_start) <= 0:
                    continue
                candidates.append(p)
            if not candidates:
                if max_conn < 10:
                    return None
                max_conn -= 1
                continue
            end = random.choice(candidates)
        
        print(f"{start} => {end} = {start.dist(end):.1f}") 
        path = [start]
        visited_c = {v for v in visited}
        total_c = total
        try:
            start_time = int(time.time())
            if not a_star(prev, start, end, path, 0):
                raise RuntimeError("no path")
            if path[-1] == very_start:
                second = cycle[1] if cycle else path[1]
                if (second - very_start) * (very_start - path[-2]) <= 0:
                    print(f"{path[-2]} => {very_start} => {second} not a good cycle")
                    raise RuntimeError("not a good cycle")
        except RuntimeError as e:
            if failed_cnt > RETRY:
                return None
            failed_cnt += 1
            visited = {v for v in visited_c}
            total = total_c
            end = None
            continue
        print(f"{len(path)} points, total dist: {total:.1f}")
        failed_cnt = 0
        assert path[0] == start
        assert not path[1] == start
        assert path[-1] == end
        path.pop()
        assert not path[-1] == end
        for p in path:
            cycle.append(p)
        if end and end == very_start:
            return cycle
        prev = path[-1]
        start = end
        end = None
        if total > dist_range[1]:
            break
        if total > dist_th:
            end = very_start
            if end in visited:
                visited.remove(end)

In [ ]:
visited = set()
for N in range(1, 10):
    print(f"######################################################################## Finding path #{N}")
    visited_copy = {v for v in visited}
    for i in range(0, len(ps)):
        if len(ps[i].ns) < MAX_CONN or ps[i] in visited:
            continue
        print(f"============================================================================ (Path #{N}) Starting from point {i}")
        start_time = int(time.time())
        total = 0
        visited.add(ps[i])
        cycle = paths(ps[i])
        if not cycle:
            visited = {v for v in visited_copy}
            continue
        cycle.append(ps[i])
        with open(f'path{N}.csv', 'w') as f:
            writer = csv.writer(f)
            for p in cycle:
                writer.writerow([p.x, p.y, p.z])
        break

In [ ]:
p1 = Point(1, 2, 3)
p2 = Point(2, 4, 6)
p3 = Point(-12, -24, -36)
print(is_line([p1, p2, p3]))
p3 = Point(-12, -22, -36)
print(is_line([p1, p2, p3]))

print(Point(467047,183200,450) - Point(467597,182247,450))